# Scraping PCPartPicker

### 👨‍💻 Autores del proyecto

* [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)
* [Alvaro López Guerrero](https://github.com/Alvalogue72)
* [Andrei Munteanu Popa](https://github.com/andu8705)

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing - `Curso 2025/2026`

### 1. Importacion de librerias
En esta celda importamos las herramientas necesarias. Usamos `undetected_chromedriver` como navegador principal para evadir la deteccion, `selenium` para la interaccion con elementos web, `pandas` para la estructura de datos, y `io.StringIO` para convertir el HTML en un formato legible para Pandas sin generar advertencias de depreciación.

In [1]:
!pip install setuptools undetected-chromedriver selenium pandas beautifulsoup4
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
from io import StringIO
import time
import random
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable


### 2. Configuracion del Driver
Esta funcion inicializa el navegador:

1. Configuramos el driver de Chrome para que se comporte como un usuario real.

2. Se establece un tiempo de carga de pagina implicito para dar margen a la conexion.

In [2]:
def setup_driver():
    options = uc.ChromeOptions()
    options.add_argument('--no-first-run')
    options.add_argument('--no-service-autorun')
    options.add_argument('--password-store=basic')
    driver = uc.Chrome(options=options, version_main=None)
    driver.implicitly_wait(10)
    return driver

### 3. Definicion de categorias
Aqui definimos un diccionario con las URLs de PCPartPicker que queremos analizar:

1. Las claves del diccionario serviran para nombrar los DataFrames posteriormente. Se pueden añadir más categorias siguiendo este mismo formato.

2. Se ha utilizado la version española (.es), pero se puede quitar si es que el usuario prefiere los precios en dolares americanos.

In [3]:
categories = {
    'cpu': 'https://es.pcpartpicker.com/products/cpu/',
    'cpu_cooler': 'https://es.pcpartpicker.com/products/cpu-cooler/',
    'gpu': 'https://es.pcpartpicker.com/products/video-card/',
    'ram': 'https://es.pcpartpicker.com/products/memory/',
    'motherboard': 'https://es.pcpartpicker.com/products/motherboard/',
    'storage': 'https://es.pcpartpicker.com/products/internal-hard-drive/',
    'cases': 'https://es.pcpartpicker.com/products/case/',
    'psu': 'https://es.pcpartpicker.com/products/power-supply/',
    'os': 'https://es.pcpartpicker.com/products/os/',
    'monitor': 'https://es.pcpartpicker.com/products/monitor/',
}

### 4. Scraping
Esta es la funcion principal. Su logica es la siguiente:
1. Navega a la URL.

2. Espera a que la tabla de productos (`#category_content`) sea visible.

3. Extrae el HTML de la tabla.

4. Utiliza `StringIO` para envolver el texto HTML y pasarlo a `pd.read_html`, que convierte automaticamente la tabla HTML en un DataFrame.

5. Usamos `BeautifulSoup` en el HTML extraido para buscar especificamente las etiquetas `<img>` dentro de las filas (`tr`).

6. Extraemos el atributo `src` de la imagen e insertamos la lista de URLs como una columna nueva en el DataFrame.

7. Realiza una limpieza basica: elimina columnas vacias (comunes en PCPartPicker por los botones de "Add") y filas nulas.

8. Gestiona la paginacion buscando el boton "Next". Si existe, hace clic y repite el proceso; si no, termina.

In [4]:
def scrape_category(driver, url, max_pages=3):
    driver.get(url)
    all_data = pd.DataFrame()
    current_page = 1
    
    while current_page <= max_pages:
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "category_content"))
            )
            
            time.sleep(random.uniform(2, 4))
            
            table_element = driver.find_element(By.CSS_SELECTOR, "table")
            table_html = table_element.get_attribute('outerHTML')
            
            html_buffer = StringIO(table_html)
            df_page = pd.read_html(html_buffer)[0]
            
            soup = BeautifulSoup(table_html, 'html.parser')  
            rows = soup.find('tbody').find_all('tr')
            
            image_urls = []
            for row in rows:
                img_tag = row.find('img')
                if img_tag and img_tag.get('src'):
                    src = img_tag.get('src')
                    if src.startswith('//'):
                        src = 'https:' + src
                    image_urls.append(src)
                else:
                    image_urls.append(None)
            
            if len(df_page) == len(image_urls):
                df_page['image_url'] = image_urls
            else:
                while len(image_urls) < len(df_page):
                    image_urls.append(None)
                df_page['image_url'] = image_urls[:len(df_page)]

            if not df_page.empty:
                all_data = pd.concat([all_data, df_page], ignore_index=True)
            
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "li.next > a")
                driver.execute_script("arguments[0].click();", next_button)
                current_page += 1
            except:
                break
                
        except Exception as e:
            break
            
    if not all_data.empty:
        all_data = all_data.dropna(axis=1, how='all')
        
    return all_data

### 5. Main Loop
En esta celda se orquesta todo el proceso:
1. Iniciamos el navegador una sola vez (para ahorrar recursos), iteramos sobre el diccionario de categorias, ejecutamos el scraping para cada una y guardamos el resultado en un diccionario de DataFrames llamado `results`.

2. Por ultimo, cerramos el navegador.

In [5]:
driver = setup_driver()
results = {}

try:
    for component_name, url in categories.items():
        df = scrape_category(driver, url, max_pages=2) 
        results[component_name] = df
        
finally:
    driver.quit()

### 6. Guardado
Aqui guardamos cada DataFrame en un archivo CSV independiente con el nombre del componente, dentro de la carpeta creada `data`.

In [6]:
if not os.path.exists('pcpartpicker/csv'):
    os.makedirs('pcpartpicker/csv')

if not os.path.exists('pcpartpicker/json'):
    os.makedirs('pcpartpicker/json')
    
for name, df in results.items():
    df.to_csv(f"pcpartpicker/csv/{name}_pspartpicker.csv", index=False, encoding='utf-8-sig')
    df.to_json(
        f"pcpartpicker/json/{name}_pspartpicker.json", 
        orient='records', 
        force_ascii=False, 
        indent=4
    )